In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import (Input, Layer, Dense, AveragePooling2D, UpSampling2D)

import tensorflow_datasets as tfds

from tensorflow.keras.applications.inception_v3 import InceptionV3

import numpy as np
import pandas as pd

from datetime import datetime
import os
import sys
import pickle
import time
import argparse
from tqdm import tqdm
import gc

sys.path.insert(0, '/vast/nj594/xai/helpers')
from evaluate import evaluate

# IMPORTANT: SET RANDOM SEEDS FOR REPRODUCIBILITY
os.environ['PYTHONHASHSEED'] = str(420)
import random
random.seed(420)
np.random.seed(420)
tf.random.set_seed(420)

In [2]:
# Load Data

data_dir = os.path.join('..', 'data')

X_val = np.load(os.path.join(data_dir, 'X_val_processed.npy'), allow_pickle=True)
X_test = np.load(os.path.join(data_dir, 'X_test_processed.npy'), allow_pickle=True)

y_val = np.load(os.path.join(data_dir, 'y_val.npy'), allow_pickle=True)
y_test = np.load(os.path.join(data_dir, 'y_test.npy'), allow_pickle=True)
y_test_fixed = np.load(os.path.join(data_dir, 'y_test_fixed.npy'), allow_pickle=True)

preds = np.load(os.path.join(data_dir, 'predictions.npy'), allow_pickle=True)
preds_discrete = np.eye(5)[preds.argmax(1)]

preds_val = np.load(os.path.join(data_dir, 'predictions_val.npy'), allow_pickle=True)
preds_discrete_val = np.eye(5)[preds_val.argmax(1)]

In [3]:
# Load Evaluator Model
print('Loading Evaluator')

eval_dir = 'evaluator-data'
evaluator_model = tf.keras.models.load_model(os.path.join(eval_dir, 'surrogate.h5'))

OPTIMIZER = tf.keras.optimizers.Adam(1e-3)
METRICS = [ 
  tf.keras.metrics.AUC(name='auroc'),
  tf.keras.metrics.AUC(curve='PR', name='auprc'),
  tf.keras.metrics.TopKCategoricalAccuracy(k=1, name='accuracy'),
]

evaluator_model.compile(
    loss='categorical_crossentropy',
    optimizer=OPTIMIZER,
    metrics=METRICS,
)

gc.collect()

Loading Evaluator


2022-05-10 12:29:01.099724: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-10 12:29:01.100219: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 32252 MB memory:  -> device: 0, name: Vega 20, pci bus id: 0000:c8:00.0


16913

In [4]:
methods = ['fastshap', 'fastshap-data', 'kernelshap', 'smoothgrad', 'gradcam', 'lime', 
           'integratedgradients', 'kernelshap_s', 'kernelshap_s-data', 'realx', 'realx-data']
results = {}
for method in ['fastshap-dkl']:
    print(method)
    results[method] = {} 
    ## Load Explanations
    if method  not in ['smoothgrad', 'integratedgradients', 'realx', 'realx-data', 'gradcam']:
        exp_file_retro = os.path.join('..', method, 'shap_values-retrospective.pkl')
        exp_file_pro = os.path.join('..', method, 'shap_values-prospective.pkl')
    elif method == 'gradcam':
        exp_file_retro = os.path.join('..', method, 'explanations.pkl')
        exp_file_pro = exp_file_retro
    else:
        exp_file_retro = os.path.join('..', method, 'explanations-retrospective.pkl')
        exp_file_pro = os.path.join('..', method, 'explanations-prospective.pkl')
        
    with open(exp_file_retro, 'rb') as f:
        explanations_retro = pickle.load(f)
    with open(exp_file_pro, 'rb') as f:
        explanations_pro = pickle.load(f)
        
    if 'realx' in method:
        print(method)
        explanations_retro += np.random.random(explanations_retro.shape) * 1e-7
        explanations_pro += np.random.random(explanations_pro.shape) * 1e-7
        
        
    ## Evaluate Explanations
    
    ### Retrospective Evaluation

    # Exclusion
    results[method]['retro_ex'] = evaluate(X_test, explanations_retro, evaluator_model, y_test_fixed, y_test_fixed, 
                                           mode = 'exclude', method = method)
    # Inclusion
    results[method]['retro_in'] = evaluate(X_test, explanations_retro, evaluator_model, y_test_fixed, y_test_fixed, 
                                           mode = 'include', method = method)
    
    ### Prospective Evaluation 

    # Exclusion
    results[method]['pro_ex'] = evaluate(X_test, explanations_pro, evaluator_model, preds_discrete, y_test_fixed, 
                                         mode = 'exclude', method = method)
    # Inclusion
    results[method]['pro_in'] = evaluate(X_test, explanations_pro, evaluator_model, preds_discrete, y_test_fixed, 
                                         mode = 'include', method = method) 
    
    print(results[method])
    
    with open('results_fixed-{}.pkl'.format(method), 'wb') as f:
        pickle.dump(results[method], f)

fastshap-dkl
100


2022-05-10 12:29:26.180735: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-05-10 12:29:26.188099: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-10 12:29:26.191872: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-10 12:29:26.194149: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-10 12:29:27.276634: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIO

99


2022-05-10 12:30:25.546836: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-10 12:30:25.550832: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-10 12:30:25.553252: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.


95


...

2022-05-10 13:08:31.479668: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-10 13:08:31.483712: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-10 13:08:31.486266: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.


0


2022-05-10 13:08:57.914550: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-10 13:08:57.918422: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-10 13:08:57.920759: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.


{'retro_ex': {'acc': {100: 0.2, 99: 0.2, 95: 0.202, 90: 0.21, 85: 0.2136, 75: 0.2148, 50: 0.2252, 25: 0.2488, 15: 0.2704, 10: 0.2968, 5: 0.3312, 1: 0.3876, 0: 0.43}, 'auroc': {100: 0.50005, 99: 0.5280263000000001, 95: 0.6374905, 90: 0.6722045, 85: 0.6864469, 75: 0.7043085, 50: 0.7204468, 25: 0.7363464, 15: 0.7508408000000001, 10: 0.7628950999999999, 5: 0.7859535999999999, 1: 0.8173984000000001, 0: 0.84102}, 'preds': {100: array([[0.71269345, 0.07373445, 0.16777113, 0.02479381, 0.0210071 ],
       [0.71269345, 0.07373445, 0.16777113, 0.02479381, 0.0210071 ],
       [0.71269345, 0.07373445, 0.16777113, 0.02479381, 0.0210071 ],
       ...,
       [0.71269345, 0.07373445, 0.16777113, 0.02479381, 0.0210071 ],
       [0.71269345, 0.07373445, 0.16777113, 0.02479381, 0.0210071 ],
       [0.71269345, 0.07373445, 0.16777113, 0.02479381, 0.0210071 ]],
      dtype=float32), 99: array([[0.7126945 , 0.07373358, 0.16777042, 0.02479379, 0.02100769],
       [0.7077213 , 0.06935478, 0.16807733, 0.026239

/vast/nj594/xai/helpers/evaluate.py:59: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  r_ = np.array([[float(p), n] for p,n in r.items()])


In [5]:
results['fastshap']

{'retro_ex': {'acc': {100: 0.2,
   99: 0.1808,
   95: 0.1452,
   90: 0.1308,
   85: 0.1284,
   75: 0.1352,
   50: 0.1708,
   25: 0.2036,
   15: 0.2228,
   10: 0.2404,
   5: 0.28,
   1: 0.3636,
   0: 0.43},
  'auroc': {100: 0.50005,
   99: 0.2938517,
   95: 0.2585059,
   90: 0.262608,
   85: 0.269838,
   75: 0.29612039999999995,
   50: 0.3777536,
   25: 0.5049036,
   15: 0.5793427999999999,
   10: 0.6299416,
   5: 0.6916209999999999,
   1: 0.7786734,
   0: 0.84102},
  'preds': {100: array([[0.71269345, 0.07373445, 0.16777113, 0.02479381, 0.0210071 ],
          [0.71269345, 0.07373445, 0.16777113, 0.02479381, 0.0210071 ],
          [0.71269345, 0.07373445, 0.16777113, 0.02479381, 0.0210071 ],
          ...,
          [0.71269345, 0.07373445, 0.16777113, 0.02479381, 0.0210071 ],
          [0.71269345, 0.07373445, 0.16777113, 0.02479381, 0.0210071 ],
          [0.71269345, 0.07373445, 0.16777113, 0.02479381, 0.0210071 ]],
         dtype=float32),
   99: array([[0.72083086, 0.07342064, 0.16

In [7]:
exp_file_retro

'../gradcam/shap_values-retrospective.pkl'

In [2]:
results = {}
for method in ['fastshap', 'fastshap-data', 'kernelshap', 'smoothgrad', 'gradcam', 'lime', 'integratedgradients',
               'fastshap-dkl', 'kernelshap_s', 'kernelshap_s-dkl', 'kernelshap_s-data', 'realx', 'realx-data']:
    with open('results_fixed-{}.pkl'.format(method), 'rb') as f:
        results[method] = pickle.load(f)

In [3]:
with open('results_fixed.pkl', 'wb') as f:
    pickle.dump(results, f)

In [9]:
results.keys()

dict_keys(['fastshap-data', 'kernelshap', 'smoothgrad', 'gradcam', 'lime', 'integratedgradients', 'kernelshap_s', 'kernelshap_s-data', 'realx', 'realx-data'])

In [7]:
with open('results_fixed.pkl', 'rb') as f:
    results = pickle.load(f)